<h2 id='part1'>A Look at the Data</h2>

In order to get a better understanding of the data we will be looking at, let's take a look at some of the characteristics of the dataset.

First, let's read in the data and necessary libraries.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df_calendar = pd.read_csv('./data/calendar.csv')

df_reviews = pd.read_csv('./data/reviews.csv')

df_listings = pd.read_csv('./data/listings.csv')

<h2 id='part1'>Question 1:</h2>
Do people favour specific seasons? 

Do get a better idea, which season is best to visit Seattle, I want to have a look, when people are mostly booking AirBnB houses. 

In [2]:
df_calendar.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [3]:
# How many listings in total? 
total_listings = df_calendar['listing_id'].nunique()


# How is the portion of available houses per month? 
availability = df_calendar.groupby('date').available.value_counts().reset_index(name='Count')
availability['Portion']=availability.Count/total_listings
availability[['Year','Month','Day']] = availability['date'].str.split(pat="-", expand = True)
Year_available = availability[(availability.available == 't') & (availability.Year=='2016')][['date','Month','Day','Portion']]
Month_available = Year_available.groupby('Month').Portion.mean().reset_index(name='Avg_Availability')
print(Month_available.sort_values(by = 'Avg_Availability'))



   Month  Avg_Availability
0     01          0.554133
6     07          0.627097
7     08          0.645051
1     02          0.662208
3     04          0.663847
5     06          0.674384
8     09          0.674402
4     05          0.675670
9     10          0.696514
2     03          0.709187
10    11          0.713986
11    12          0.735573


The most preferred seasons to visit Seattle seem to be beginning of the year in January and Feruary or in summer in July and August.

Let's have a look into the different price ranges between (January,February) and (July, August).

<h2 id='part1'>Question 2:</h2>
In which of these two season can I get a cheeper house?  

In [4]:
df_calendar[['Year','Month','Day']] = df_calendar['date'].str.split(pat="-", expand = True)

df_beginning = df_calendar[(df_calendar.Year == '2016') & ((df_calendar.Month == '01') | (df_calendar.Month == '02')) & (df_calendar.available == 't')]
df_summer = df_calendar[(df_calendar.Year == '2016') & ((df_calendar.Month == '07') | (df_calendar.Month == '08')) & (df_calendar.available == 't')]

# What is the average price of the available houses in beginning of year versus summer?
#print(df_beginning.head())
prices_janfeb = df_beginning.groupby('price')['listing_id'].nunique().reset_index(name='no_listings')
prices_janfeb['price_number'] = prices_janfeb['price'].str[1:-3].replace('1,000','1000').astype(int)
prices_janfeb = prices_janfeb.sort_values('price_number', ascending = True)

#print(prices_janfeb.sort_values('no_listings', ascending = False)[:10])
#print(prices_janfeb['price_number'].value_counts(bins=15))

bins = [0, 50, 100, 150, 200, 1500]
prices_janfeb['binned'] = pd.cut(prices_janfeb['price_number'], bins)
print(prices_janfeb.groupby('binned')['no_listings'].sum())

binned
(0, 50]        1096
(50, 100]      4394
(100, 150]     3399
(150, 200]     1290
(200, 1500]     994
Name: no_listings, dtype: int64


In [5]:
df_summer = df_calendar[(df_calendar.Year == '2016') & ((df_calendar.Month == '07') | (df_calendar.Month == '08')) & (df_calendar.available == 't')]

# What is the average price of the available houses in beginning of year versus summer?
#print(df_beginning.head())
prices_julaug = df_summer.groupby('price')['listing_id'].nunique().reset_index(name='no_listings')
prices_julaug['price_number'] = prices_julaug['price'].str[1:-3].replace('1,000','1000').replace('1,199','1199').replace('1,240','1240').replace('1,250','1250').astype(int)
prices_julaug = prices_julaug.sort_values('price_number', ascending = True)

bins = [0, 50, 100, 150, 200, 1500]
prices_julaug['binned'] = pd.cut(prices_julaug['price_number'], bins)
print(prices_julaug.groupby('binned')['no_listings'].sum())


binned
(0, 50]         225
(50, 100]      1487
(100, 150]     1119
(150, 200]      964
(200, 1500]    2060
Name: no_listings, dtype: int64


Generally in winter are more cheep houses available than in summer. I can select out of 1096 houses for less than 50$. In summer I can select out of 225 houses for less than 50$. 

I decide to visit Seattle in winter.


<h2 id='part1'>Question 3:</h2>
Which of that cheep houses are available for 2 weeks at least?

In [98]:
#Select only cheep houses first
df_beginning['price_number'] = df_beginning['price'].str[1:-3].replace('1,000','1000').astype(int)
df_cheep = df_beginning[df_beginning['price_number'] <=50]

# which houses are available for 2 weeks?
df_cheep = df_cheep.sort_values(['listing_id', 'date'])
df_cheep_grouped = df_cheep.groupby(['listing_id','Month'])['Day'].agg(['unique']).reset_index()
df_cheep_grouped_days = df_cheep.groupby(['listing_id','Month'])['Day'].agg(['nunique']).reset_index()
df_cheep_grouped['nunique'] = df_cheep_grouped_days['nunique']


# Finde die Zeilen, wo 14 aufeinander folgende Tage enthalten sind:

# Function to count ones in array
def count_numbers(numbers):
    """
    Input: [0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0]
    Output: [2, 4, 7, 1]
    """
    results = []
    counter = 0
    if all(p == 1 for p in numbers):
        results.append(len(numbers))
    else:
        for number in numbers:
            if number == 1:
                counter += 1
            elif (number != 1) and counter:
                results.append(counter)
                counter = 0
    return results

"""
Determine consecutive days and count nights
"""
two_weeks_available = []

for i in enumerate(df_cheep_grouped["unique"]):
    array = i[1].astype(int)

    day_diff = []
    for val in range(len(array)-1):
        #calculate difference between numbers to determine number of nights
        day_diff.append(array[val+1] - array[val])

    """ 
    Use function for counting ones to determine how many nights are between consecutive days
    if element2-element1 = 1 -> 1 night in between and then it is counted
    """    
    count_consecutive_nights = count_numbers(day_diff)
   

    """ 
    Return index if house is available for at least 13 consecutive nights (Output: list of indexes from initial dataframe)
    """
    if np.any(np.asarray(count_consecutive_nights) >= 13):
        two_weeks_available.append(i[0])
    

available_houses = df_cheep_grouped.iloc[two_weeks_available]
print(available_houses['listing_id'].unique())



[  149489   226677   444221   447523   486344   671828   677266   815017
   817115  1009838  1142039  1305009  1328159  1340668  1511620  1520449
  1520501  1520533  1534622  1541705  1856253  1901706  1925661  2187906
  2263643  2605975  2642359  2729077  2986056  3005854  3139972  3245876
  3246706  3282760  3403638  3404557  3424114  3424242  3535171  3544964
  3637180  3656508  3666970  3732076  3732094  3732103  3793790  3811828
  3811872  3811930  3811955  3844087  3876097  3888924  3888986  3889050
  3889280  3889299  3889330  3916050  3939683  3953365  3977450  3994601
  3994634  4126284  4288801  4374326  4384095  4468524  4494947  4559222
  4581256  4639040  4701205  4710866  4710869  4718820  4725935  4735761
  4760181  4825472  4832481  4910140  4941356  4995903  5002964  5031285
  5031383  5062445  5067177  5078244  5310503  5324459  5640722  5873892
  5892185  5950957  5969872  6107518  6117473  6119821  6249164  6317449
  6403104  6591894  6629657  6657574  6831239  6888

c:\Users\Nadine.Puetzer1\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


As a result I got a list of 218 houses, that I can visit in the most favoured season, in winter, for a price less than 50$ per night and are available for at least 13 consecutive nights.